# An Example of Data Processing and Model Updating

This document illustrates a typical data processing steps and the basic model updating using R and Stan.

The data we use are from the Coldwater Creek.  Initiall, only 8 monthly averages were available. We fit the hockey-stick model, which yields a highly uncertain estimate of the retentopn capacity.  The model was used to estimate the necessary sample size for a more stable model.  WIht data from subsequent monitoring, we updated the model.  Both data sets were stored in csv format.

The following tab connects user Google Drive to the cloud

In [1]:
%load_ext rpy2.ipython
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


We now setup the cloud R system.  Change the line `base <- "..."` to your folder system, assuming that you have a subfolder `Data` where R can find necessary input data. This cell also includes some functions needed to run the model.

In [0]:
%%R
base <- "/content/drive/My\ Drive/Song\'s\ Lab/temp_ms/OldWomanCreek/R/HockeyStick"
dataDIR <- paste(base, "Data", sep="/")
plotDIR <- paste(base, "Figs", sep="/")

system("add-apt-repository -y ppa:marutter/rrutter")
system("add-apt-repository -y ppa:marutter/c2d4u")
system("apt-get update")
system("apt install -y r-cran-rstan")

to2 <- function(x){
  ## conver a single digit integer to 0x
  return(ifelse (x<10, paste("0",x, sep=""), as.character(x)))
}

hockey <- function(x, beta0, beta1, phi) return(beta0+beta1*(x-phi)*(x>phi))

hockey_smooth <- function(x, beta0, beta1=0, delta, phi, theta=NULL)
{
  if (is.null(theta)) theta=0.01*diff(range(x))
  return(beta0 + beta1 * (x-phi) +
           delta * theta * log1p(exp((x-phi)/theta)))
}

movingAverage <- function(x, n=1, centered=FALSE) {
  if (centered) {
    before <- floor  ((n-1)/2)
    after  <- ceiling((n-1)/2)
  } else {
    before <- n-1
    after  <- 0
  }
  
  ## Track the sum and count of number of non-NA items
  s     <- rep(0, length(x))
  count <- rep(0, length(x))
  
  # Add the centered data
  new <- x
  # Add to count list wherever there isn't a
  count <- count + !is.na(new)
  # Now replace NA_s with 0_s and add to total
  new[is.na(new)] <- 0
  s <- s + new
  
  # Add the data from before
  i <- 1
  while (i <= before) {
    # This is the vector with offset values to add
    new   <- c(rep(NA, i), x[1:(length(x)-i)])
    
    count <- count + !is.na(new)
    new[is.na(new)] <- 0
    s <- s + new
    
    i <- i+1
  }
  
  # Add the data from after
  i <- 1
  while (i <= after) {
    # This is the vector with offset values to add
    new   <- c(x[(i+1):length(x)], rep(NA, i))
    
    count <- count + !is.na(new)
    new[is.na(new)] <- 0
    s <- s + new
    
    i <- i+1
  }
  
  # return sum divided by count
  s/count
}

line.plots <- function(est, se, Ylabel, Xlab, yaxis=2, Xlim=NULL,
                       rscale=F, At=NULL, Lab=NULL, V=0){
  if (is.null(Xlim)) Xlim <- range(c(est+2*se, est-2*se))
  n <- length(est)
  if(n != length(se))stop("lengths not match")
  plot(1:n, 1:n, xlim=Xlim, ylim=c(0.75, n+0.25),
       type="n", axes=F, xlab=Xlab, ylab="")
  if (rscale)
    axis(1, at=At, labels=Lab)
  else
    axis(1)
  axis(yaxis, at=1:n, labels=Ylabel, las=1)
  segments(y0=1:n, y1=1:n, x0=est-2*se, x1=est+2*se)
  segments(y0=1:n, y1=1:n, x0=est-1*se, x1=est+1*se, lwd=2.5)
  points(est, 1:n)
  if (!is.null(V)) abline(v=V, col="gray")
  invisible()
}

packages<-function(x, repos="http://cran.r-project.org", ...){
  x<-as.character(match.call()[[2]])
  if (!require(x,character.only=TRUE)){
    install.packages(pkgs=x, repos=repos, ...)
    require(x,character.only=TRUE)
  }
}

packages(tidyverse)
packages(readr)

packages(reshape2)
packages(rv)
packages(arm)
packages(lattice)
#packages(tikzDevice)
packages(car)
packages(formatR)
packages(rstan)

rstan_options(auto_write = TRUE)
options(mc.cores = min(c(parallel::detectCores(), 8)))

nchains <-  min(c(parallel::detectCores(), 8))
niters <- 100000
nkeep <- 2500
nthin <- ceiling((niters/2)*nchains/nkeep)


## Importing Data

In [0]:
%%R
cwc <- read_csv(paste(dataDIR, "cwcdata.csv", sep="/"))  ## initial data
names(cwc) <- c("week","site","flow","height","restime", "Date", "NO3_in", "TP_in", "DRP_in", "TSS_in", "NO3_out", "TP_out", "DRP_out", "TSS_out")
## rename columns
cwc$RDate <- as.Date(cwc$Date, format="%m/%d/%y") ## R dates

cwc_temp <- cwc[,c("flow","NO3_in","TP_in","DRP_in","TSS_in","NO3_out","TP_out","DRP_out","TSS_out","RDate")] ## relevant columns
cwc_temp$TPLoading <- cwc$TP_in*cwc$flow * 0.001989611 ## converting units to tons per year -- flow in GPM and conc in mg/L
cwc_temp$DRPLoading <- cwc$DRP_in*cwc$flow * 0.001989611  ## tons per year
cwc_temp$NO3Loading <- cwc$NO3_in*cwc$flow * 0.001989611  ## tons per year
cwc_temp$Month <- format(cwc$RDate, format="%m/%y")
cwc_month <- melt(cwc_temp, id=c("RDate", "Month"))
  
cwc_month <- dcast(cwc_month, Month~variable, mean, na.rm=T)  ## monthly averages
xyplot(TP_out ~ log(TPLoading), data=cwc_month) ## 
xyplot(DRP_out ~ log(DRPLoading), data=cwc_month)


In [0]:
Updated data

In [0]:
%%R
## Updating -- New data were made available near the end of the project
cwc_flow <- read.csv(paste(dataDIR, "CWC_Flow.csv", sep="/"))
names(cwc_flow) <- c("Date", "Site", "Flow", "tau")
cwc_flow$RDate <- as.Date(cwc_flow$Date, format="%m/%d/%Y")
cwc_flow$Month <- format(cwc_flow$RDate, format="%m/%y")

cwc_tp <- read.csv(paste(dataDIR, "CWC_TP2.csv", sep="/"))
names(cwc_tp) <- c("Date", "Season", "Site", "TP")
cwc_tp$RDate <- as.Date(cwc_tp$Date, format="%m/%d/%y")
cwc_tp$Month <- format(cwc_tp$RDate, format="%m/%y")

cwc_tp_in <- cwc_tp %>% filter(Site==levels(Site)[1])
cwc_tp_out <- cwc_tp %>% filter(Site==levels(Site)[2])

flow_mean <- group_by(cwc_flow, Month) %>%dplyr::summarise(fmean=mean(Flow, na.rm=T))
tp_in_mean <- group_by(cwc_tp_in, Month) %>%dplyr::summarise(TP_in=mean(TP, na.rm=T))
tp_out_mean <- group_by(cwc_tp_out, Month) %>%dplyr::summarise(TP_out=mean(TP, na.rm=T))

cwc_monthly <- left_join(flow_mean, tp_in_mean, by="Month")
cwc_monthly <- left_join(cwc_monthly, tp_out_mean, by="Month")
cwc_monthly$TPload <- cwc_monthly$fmean*cwc_monthly$TP_in * 0.001989611
## tons per year

ggplot(cwc_monthly, aes(x=TPload, y=TP_out)) +
  geom_point()+
  scale_x_log10()+
  scale_y_log10()+
  xlab("TP Loading (ton/yr)")+ylab("TP Concentration (mg/L)")


# Fitting the Hockey-Stick Model

The Bayesian updating of the hockey-stick model includes the following steps:

1. Using an existing model to extract prior information. In this case, we initially used the model from the Old Woman Creek.  The following R function extracts the prior information.


In [0]:
%%R
## using the hierarchcial mdoel for the prior

prior <- function(fit, b0="B0", de="De", ph="Ph",
                  s0="sigma0",sD="sigmaD",sP="sigmaP",
                  n0=20, setn0=F){
    fit2SNprior <- rvsims(as.matrix(as.data.frame(rstan::extract(fit, permute=T))))
    
    tmp <- summary(fit2SNprior[names(fit2SNprior)==b0])
    Eb0 <- tmp$mean
    Vb0 <- tmp$sd^2
    
    tmp <- summary(fit2SNprior[names(fit2SNprior)==de])
    EDe <- tmp$mean
    VDe <- tmp$sd^2
    
    tmp <- summary(fit2SNprior[names(fit2SNprior)==ph])
    EPh <- tmp$mean
    VPh <- tmp$sd^2
    
    tmp <- summary(fit2SNprior[names(fit2SNprior)==s0]^2)
    Esigma0 <- tmp$mean
    Vsigma0 <- tmp$sd^2
    
    tmp <- summary(fit2SNprior[names(fit2SNprior)==sD]^2)
    EsigmaD <- tmp$mean
    VsigmaD <- tmp$sd^2
    
    tmp <- summary(fit2SNprior[names(fit2SNprior)==sP]^2)
    EsigmaP <- tmp$mean
    VsigmaP <- tmp$sd^2

    if (setn0) {
        alpha0 <- n0+1
        alphaD <- n0+1
        alphaP <- n0+1
    } else {
        alpha0 <- 2+Esigma0/Vsigma0
        alphaD <- 2+Esigma0/VsigmaD
        alphaP <- 2+Esigma0/VsigmaP
    }
    beta0 <- Esigma0*(alpha0-1)
    betaD <- EsigmaD*(alphaD-1)
    betaP <- EsigmaP*(alphaP-1)
    lambda0 <- Esigma0/Vb0
    lambdaD <- EsigmaD/VDe
    lambdaP <- EsigmaP/VPh
    ## limiting alpha + beta to be less than 1000
    while (alpha0+beta0 > 1000){
      alpha0 <- alpha0/10
      beta0 <- beta0/10
    }
    while (alphaD+betaD > 1000){
      alphaD <- alphaD/10
      betaD <- betaD/10
    }
    while (alphaP+betaP > 1000){
      alphaP <- alphaP/10
      betaP <- betaP/10
    }
    return(list(m0=Eb0, mD=EDe, mP=EPh,
                lmbd0=lambda0, lmbdD=lambdaD, lmbdP=lambdaP,
                al0=alpha0, alP=alphaP, alD=alphaD,
                bt0=beta0, btP=betaP, btD=betaD))
}



2. Revised the hockey stick model to use informative prior for the hyper-parameters.

The Stan model in the below R code chunk includes a hyper-parameter for each model coefficients:


In [0]:
%%R
stan_model3 <- "
	  data{
	  int N; //the number of observations
	  vector[N] y; //the response
	  vector[N] x; 

	  real theta;
          real beta1;

          real m0;
          real mD;
          real mP;

          real lmbd0;
          real lmbdD;
          real lmbdP;

          real al0;
          real alP;
          real alD;

          real bt0;
          real btP;
          real btD;

	}
	parameters {
	  real beta0; //the regression parameters
	  real<lower=0> delta;
	  real phi; //change point

	  real<lower=0> sigma;

    real mu0;
    real muD;
    real muP;

	  real<lower=0> sigma0sq;
	  real<lower=0> sigmaDsq;
	  real<lower=0> sigmaPsq;
	}
	transformed parameters {
	  vector[N] mu;
	  real<lower=0> sigma0;
	  real<lower=0> sigmaD;
	  real<lower=0> sigmaP;
	  
    sigma0 = sqrt(sigma0sq);
    sigmaD = sqrt(sigmaDsq);
    sigmaP = sqrt(sigmaPsq);
	  for (i in 1:N)
	    mu[i] = beta0 + beta1 * (x[i]-phi) +
		    delta * theta *
			    log1p(exp((x[i]-phi)/theta));
	}
	model {  
	  sigma ~ cauchy(0, 1)  ;
	  sigma0sq ~ inv_gamma(al0, bt0);
	  sigmaDsq ~ inv_gamma(alD, btD);
	  sigmaPsq ~ inv_gamma(alP, btP);
       
    mu0 ~ normal(m0, sigma0/sqrt(lmbd0));
    muD ~ normal(mD, sigmaD/sqrt(lmbdD));
    muP ~ normal(mP, sigmaP/sqrt(lmbdP));
          
	  phi ~ normal(muP, sigmaP); 
	  beta0 ~ normal(mu0, sigma0); 
	  delta ~ normal(muD, sigmaD); 

	  y ~ normal(mu, sigma);
	}
"


3. Processing input data to create Stan model input and initial data files. The function in the below R code chunk takes input of (1) data file name, (2) x (TP loading) column name, (3) y (TP effluent concentration) column name, (4) subsect of the data, (5) number of chains for the MCMC run, (6) whether the x variable is standardized, (7) whether informative prior is used, and (8) the prior.


In [0]:
%%R
stan.in2 <- function(infile=owc_daily, x="maTPLD05", y="TP_ef",
                     grp=owc_daily$Quarter=="Q1", n.chains=4,
                     stdz=T,info=F, prrs = NULL){
    if (info & is.null(prrs)) stop("Need informative priors")
    infile <- infile[grp,]
    keep <- (infile[,x] > 0) & (infile[,y] >0)
    infile <- infile[keep & !is.na(keep),]
    x <- log(infile[,x])
    if (stdz){
      xmu <- mean(x, na.rm=T)
      xsd <- sd(x, na.rm=T)
      x <- (x - xmu)/xsd
    } else {
      xmu <- 0
      xsd <- 1
    }
    y <- log(infile[,y])
    n <- dim(infile)[1]
    if (info){
        m0 = prrs$m0
        mD = prrs$mD
        mP = prrs$mP
        lmbd0=prrs$lmbd0
        lmbdD=prrs$lmbdD
        lmbdP=prrs$lmbdP
        al0=prrs$al0
        alP=prrs$alP
        alD=prrs$alD
        bt0=prrs$bt0
        btP=prrs$btP
        btD=prrs$btD
    }else{
        m0 = 0
        mD = 0
        mP = 0
        lmbd0=1
        lmbdD=1
        lmbdP=1
        al0=2
        alP=2
        alD=2
        bt0=2
        btP=2
        btD=2
    }

    s0 <- sqrt(bt0/(al0-1))
    sD <- sqrt(btD/(alD-1))
    sP <- sqrt(btP/(alP-1))
    
    inits <- list()
    if (stdz) theta <- 0.04
    else theta <- 0.01*diff(range(x))
    bugs.data <- list(N=n, y=y, x=x,
                      theta=theta, beta1=0,
                      m0 = m0,  mD = mD, mP = mP,
                      lmbd0=lmbd0, lmbdD=lmbdD, lmbdP=lmbdP,
                      al0=al0, alP=alP, alD=alD,
                      bt0=bt0, btP=btP, btD=btD )
    for (i in 1:n.chains)
	inits[[i]] <- list(beta0=rnorm(1, m0, s0),
	                   delta=rnorm(1,mD,sD),
			   phi=runif(1, range(x)[1], range(x)[2]),
			   sigma=runif(1), sigmaPsq=runif(1), sigmaDsq=runif(1),
                           sigma0sq=runif(1),
                           mu0=rnorm(1, m0,s0), 
			   muD=rnorm(1, mD, sD),
                             muP=rnorm(1, mP,sP))
    para <- c("beta0", "delta", "phi","sigma",
              "mu0","muD","muP", "sigma0","sigmaD","sigmaP"
              )
    return(list(para=para, data=bugs.data, inits=inits,n.chains=n.chains,
                mux=xmu, sdx=xsd))
}



4. Loading and processing prior information


In [0]:
%%R
load(paste(base, "owcTPma05SeasonSTDZ.RData", sep="/"))  ## the seasonal OWC model

prr <- prior(fit2)  ## fit2 is the OWC seasonal model
tmp <- cwc_month$flow!=0

input.to.stan <- stan.in2(infile=cwc_month[tmp,],
                          x="TPLoading", y="TP_out", 
                          grp=rep(T, sum(tmp)), info=T, prrs=prr,
                          n.chains=nchains)
muxCWC <- input.to.stan$mux
sdxCWC <- input.to.stan$sdx
thetaCWC <- input.to.stan$data$theta



5. Fitting the Bayesian model using Stan.  


In [0]:
%%R
fit <- stan_model(model_code = stan_model3)  ## compile model (it may take a while)
## updating/sampling (it may take a while)
fitCWC <- sampling(fit, data = input.to.stan$data, ##init=input.to.stan$inits,
                  pars = input.to.stan$para,
                  iter=niters, thin=nthin, chains=input.to.stan$n.chains,
                  control = list(adapt_delta = 0.99, max_treedepth=20))

## rescale the loading related variables:
fitcoefCWC <- rvsims(as.matrix(as.data.frame(rstan::extract(fitCWC, permute=T))))
fitcoefCWC$delta <- fitcoefCWC$delta/sdxCWC
fitcoefCWC$phi <- muxCWC+fitcoefCWC$phi*sdxCWC
fitcoefCWC_sum <- summary(fitcoefCWC)


6. Processing output: Stan output are Monte Carlo simulation samples. We use the R package `rv` to process the output.


In [0]:
%%R
df_cwc <- dplyr::select(cwc_month, TPLoading, TP_out) %>% filter(!is.na(TP_out)) %>% arrange(TPLoading)
df_cwc <- cbind(df_cwc, summary(hockey(x=log(df_cwc$TPLoading), fitcoefCWC[1], fitcoefCWC[2], fitcoefCWC[3])))
names(df_cwc) <- c("TPLoading","TP_out","mean","sd","X1",       "X2.5",      "X25",        "X50",         "X75",       "X97.5",        "X99","sims")
p <- ggplot(data=df_cwc, aes(x=TPLoading, y=TP_out))+geom_point(color="blue", size=0.75)+
  scale_x_continuous(name = "TP Loading (ton/yr)", breaks=c(1,2, 5,7), 
                       labels=c("1","2","5", "7"), trans="log")+
    scale_y_continuous(name = "TP Effluent Concentration (mg/L)", 
                       breaks=c(0.2,0.5,0.75,1), labels=c("0.2","0.5", "0.75", "1"), trans="log") + 
    geom_ribbon(aes(x=TPLoading, ymin=exp(X2.5), ymax=exp(X97.5)), alpha=0.3) + 
    geom_line(aes(x=TPLoading, y=exp(mean)), lwd=1.25)

print(p)



## Updating with New Data

With new data made available, the updating process is slightly modified by using the posterior from the previous iteration


In [0]:
%%R
prr <- prior(fitCWC, b0="mu0", de="muD", ph="muP")

input.to.stan <- stan.in2(infile=as.data.frame(cwc_monthly),
                          x="TPload", y="TP_out", 
                          grp=rep(T, dim(cwc_monthly)[1]), info=T, prrs=prr,
                          n.chains=nchains)
muxCWC_n <- input.to.stan$mux
sdxCWC_n <- input.to.stan$sdx
thetaCWC_n <- input.to.stan$data$theta

fit <- stan_model(model_code = stan_model3)
fitCWC_n <- sampling(fit, data = input.to.stan$data, 
##                     init=input.to.stan$inits,
                     pars = input.to.stan$para,
                     iter=niters, thin=nthin, chains=input.to.stan$n.chains,
                     control = list(adapt_delta = 0.99, max_treedepth=20))


fitcoefCWC_n <- rvsims(as.matrix(as.data.frame(rstan::extract(fitCWC_n, permute=T))))
fitcoefCWC_n$delta <- fitcoefCWC_n$delta/sdxCWC_n
fitcoefCWC_n$phi <- muxCWC_n+fitcoefCWC_n$phi*sdxCWC_n
fitcoefCWC_sum_n <- summary(fitcoefCWC_n)

save(fitCWC_n, fitcoefCWC_n, muxCWC_n, sdxCWC_n, thetaCWC_n, file="ColdCreek_STDZ_n.RData")
## load("ColdCreek_STDZ_n.RData")

df_cwc_n <- dplyr::select(cwc_monthly, TPload, TP_out) %>% filter(!is.na(TP_out)) %>% arrange(TPload)
df_cwc_n <- cbind(df_cwc_n, summary(hockey(x=log(df_cwc_n$TPload), fitcoefCWC_n[1], fitcoefCWC_n[2], fitcoefCWC_n[3])))
names(df_cwc_n) <- c("TPload","TP_out","mean","sd","X1",       "X2.5",      "X25",        "X50",         "X75",       "X97.5",        "X99","sims")
p <- ggplot(data=df_cwc_n, aes(x=TPload, y=TP_out))+geom_point(color="blue", size=0.75)+ 
    geom_ribbon(aes(x=TPload, ymin=exp(X2.5), ymax=exp(X97.5)), alpha=0.3) + 
    geom_line(aes(x=TPload, y=exp(mean)), lwd=1.25)+
    scale_x_continuous(name = "TP Loading (ton/yr)", breaks=c(0.5, 1,2, 5,7), 
                       labels=c("0.5","1","2","5", "7"), trans="log")+
    scale_y_continuous(name = "TP Effluent Concentration (mg/L)", 
                       breaks=c(0.1,0.2,0.5,0.75,1), labels=c("0.1","0.2","0.5", "0.75", "1"), trans="log") 

print(p)

